In [ ]:
!pip install nltk
!pip install spacy
!python -m spacy download en_core_web_lg
!python -m spacy link en_core_web_lg en --force

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=41f5fab301c436a4cc54287d94aa6cc03f2d05f7112bbfe1e0a35f5f92495d6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-_m4636wi/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from gensim.models.phrases import Phraser
from gensim.models import Phrases
from nltk.corpus import stopwords
from google.colab import drive
import pandas as pd
import numpy as np
import warnings
import spacy
import json
import nltk
import re
import os

nltk.download('stopwords')
drive.mount('/content/drive')
dir = '/content/drive/MyDrive/Panasonic/'
os.chdir(dir)
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


In [ ]:
nlp = spacy.load('en')

In [ ]:
def clean(s):
    # lowercase
    s = s.lower()
    
    # remove chars that are:
    #   ^\w: NOT word char (not alphanumeric)
    #   ^\s: NOT space
    #   ^. : NOT period
    #   |_ : OR underscore
    s = re.sub(r'[^.\w\s]|_', ' ', s)
    
    # remove excess spaces
    s = re.sub(' +', ' ', s)
    s = '\n'.join(line.strip().rstrip('.') for line in s.split('\n'))

    return s


def form_sentences(s, min_tokens=3, min_token_len=1):
    sentences = []

    # split by '.' and '\n' to get sentences
    s = s.replace('. ', '\n').replace('.\n', '\n')
    for sent in s.split('\n'):
        # tokens = space separated words in a sentence
        # e.g., 'this is an apple' has 4 tokens
        # Condition 1) a sentnece must have at least min_tokens (3) tokens
        # Condition 2) each token must be of min_token_len (1)
        tokens = [t for t in sent.split() if len(t) >= min_token_len]
        if len(tokens) >= min_tokens:
            sentences.append(' '.join(tokens))

    return sentences

In [ ]:
def is_company(ent):
    if ent.label_ != 'ORG':
        return False
    for c in ent.text:
        if (not c.isalnum()) and (c not in ['.', ' ']):
            return False

    if ent.text.lower() in ['svg', 'png', 'jpg', 'jpeg', 'pdf', 'ico', 'icns',
                            'clipart', 'shutterstock', 'favicon']:
        return False

    return True

In [ ]:
exempt = ['industry', 'group', 'company', '.co', 'inc.', 'ltd', 'ltd.', 'robotics', ]
def contains_part(a, b):
    a = a.split()
    b = b.split()
    for sub_a in a:
        if sub_a in b and sub_a not in exempt:
            return True
    for sub_b in b:
        if sub_b in a and sub_b not in exempt:
            return True
    return False

In [ ]:
def clean_company(s):
    s = s.split()
    for rem in ['logo', 'flag', 'icon']:
        while rem in s:
            s.remove(rem)

    s = ' '.join(s)

    if s.isdigit() or s in ['en', 'fr', 'de', 'zh', 'zh hant', 'ko', 'es', 'about', 'mission', 'testimonials', 'projects', 'products', 'solutions', 'bin', '3d']:
        return ''

    return s

In [ ]:
def find_common(a, b):
    a = a.split()
    b = b.split()
    common = []
    for sub_a in a:
        if sub_a in b:
            common.append(sub_a)
    for sub_b in b:
        if sub_b in a:
            common.append(sub_b)
    
    if common:
        return ' '.join(list(set(common)))
    return ''

In [ ]:
def find_ext(path):
    lower = path.lower()
    if path:
        for ext in ['.png', '.jpg', '.jpeg']:
            if lower.endswith(ext):
                return ext
            elif ext in lower:
                for idx in [x.end() for x in re.finditer(ext, lower)]:
                    if idx < len(lower) and not lower[idx].isalnum():
                        return ext

    return ''

In [ ]:
def get_url_tail(s):
    ext = find_ext(s)
    s = s.split(ext)[0].split('/')[-1]
    if s:
        s = s.lower()
        s = s.replace('%20', ' ')
        s = re.sub(r'[^.\w\s]|_', ' ', s)
        s = [w for w in s.split() if w.replace('.', '').isalpha()]

        return ' '.join(s)
    
    return ''    

In [ ]:
results_max = 1000
results_min = 50
clients = []

tier1 = 1
tier2 = 1
tier3 = 1

def json_to_clients_1(file):
    with open(file) as f:
        data = json.load(f)
        for key in list(data.keys()):
            obj = data[key]
            found = False

            ## Tier 1                                
            ## - Alt text                            
            ## - Header of 1st reverse search result 
            ## - Body of 1st revese search result    
            if tier1:
                if obj['alt']:
                    clients.append(clean_company(obj['alt']))
                    print('[1]', clients[-1], '|', obj['url'])
                    break
                else:
                    if obj['header']:
                        header_doc = nlp(obj['header'])
                        for ent in header_doc.ents:
                            if ent.text not in clients and ent.label_ == 'ORG':
                                print('[1]', ent.text, '|', obj['url'])
                                clients.append(ent.text)
                                found = True
                                break
                        if not found:
                            print('[1]', obj['header'], '|', obj['url'])
                            clients.append(obj['header'])
                            found = True
                    if not found and obj['body']: 
                        body_doc = nlp(obj['body'])
                        for ent in body_doc.ents:
                            if ent.text not in clients and ent.label_ == 'ORG':
                                print('[1***]', ent.text, '|', obj['url'])
                                clients.append(ent.text)
                                found = True
                                break

            ## Tier 2 
            ## URL tail     = E.g., 'www.intermodalics/wpcontent/audi.png' will yield 'audi'
            ## Search value = reverse search's search bar value
            ## Find common words between both
            if tier2 and not found:
                tail = obj['url_tail']
                common = find_common(clean_company(obj['search_value']), clean_company(tail))
                if common and common not in clients:
                    clients.append(common)
                    found = True
                    print('[2]', common, '|', obj['url'])
            
            ## Tier 3 
            ## Use URL tail directly as client
            if tier3 and not found:
                clients.append(clean_company(tail))
                print('[3]', tail, '|', obj['url'])

    return clients

In [ ]:
def exclude_words(words, exclude_list):
    if isinstance(words, str):
        return ' '.join([word for word in words.split() if word.lower() not in exclude_list])
    elif isinstance(words, list):
        return [word for word in words if word.lower() not in exclude_list]

In [ ]:
exclude = stopwords.words('english')
common_words = ['logo', 'logos', 'hd', 'png', 'svg', 'jpg', 'jpeg', 'home', 'homepage', 'website', 'hd', 'free', 'vector', 'dot', 'gifs', 'vertical', 'language', 'download', 'white', 'black', 'color', 'rgb', 'cmyk', 'index', 'rectangle', 'transparent', 'square', 'header', 'footer', 'screenshot']
data_keys = ['url_tail', 'header', 'search_value', 'alt']
def json_to_clients_2(file):
    results = {
        'url': [],
        'page': [],
        'alt': [],
        'url_tail': [],
        'common': []
    }
    company = file.split('/')[-1].split('-clients')[0]
    exclude_list = exclude + [company.lower()]

    box_hor = '+-'+ len(company)*'-' + '-+'
    print(box_hor)
    print(f'| {company} |')
    print(box_hor)

    with open(file) as f:
        data = json.load(f)
        for logo in list(data.keys()):
            data[logo]['url_tail'] = get_url_tail(data[logo]['url'])

            # get bigrams and trigrams
            sentence_stream = []

            for data_key in data_keys:
                data_value = data[logo][data_key]
                if data_value:
                    data_value = data_value.replace('-', '_').replace(' ', '_')
                    sentence_stream.append(exclude_words(data_value.split('_'), common_words))
            
            bigrams = Phrases(sentence_stream, min_count=1, threshold=1, delimiter=b'_')
            bigram_text = [bigrams[sentence] for sentence in sentence_stream]

            trigrams = Phrases(bigram_text, min_count=1, threshold=1)
            trigram_text = [trigrams[sentence] for sentence in bigram_text]

            for data_key in data_keys:
                data_value = data[logo][data_key]
                if data_value:
                    data[logo][data_key] = ' '.join(trigram_text.pop(0))

            # 1) find most common words
            # 2) extend abbreviations to full forms
            text = []
            freq = {}
            for key in data_keys:
                text += data[logo][key].split()

            text = [word for word in text if len(word)>=2]
            text = exclude_words(text, exclude_list)

            # create freq list
            for w in sorted(list(set(text)), key=len, reverse=True):        
                word = w
                freq[word] = 0
                for cmp in text:
                    if word == cmp:
                        freq[word] += 1
                    elif len(cmp) > len(word) and word == cmp[:len(word)]:
                        if cmp in freq:
                            freq[cmp] += freq[word] + 1
                        else:
                            freq[cmp] = freq[word] + 1
                        freq.pop(word)
                        word = cmp

            client = ''
            if freq:
                max_freq = max(list(freq.values()))
                for token in freq:
                    if freq[token] == max_freq and token.lower() not in client.lower().split():
                        client += token + ' '

            results['url'].append(data[logo]['url'])
            results['url_tail'].append(exclude_words(data[logo]['url_tail'], exclude_list))
            results['alt'].append(exclude_words(data[logo]['alt'], exclude_list))
            results['page'].append(data[logo]['page'])
            results['common'].append(client.strip())

    # sometimes alt text contains irrelevant decorators like index_google, index_samsung. we want to remove the 'index'
    for tier in ['alt', 'url_tail', 'common']:
        client_freq = {} 
        clients = results[tier]
        
        for client in clients:
            for word in client.split():
                if word not in client_freq:
                    client_freq[word] = 0
                client_freq[word] += 1
        
        for freq_word in list(client_freq.keys()):
            if client_freq[freq_word] < max(2, len(list(data.keys()))//2):
                client_freq.pop(freq_word)
        
        if client_freq:
            for freq_word in client_freq:
                print('Duplicate:', freq_word)
                for i in range(len(clients)):
                    if freq_word in clients[i]:
                        clients[i] = clients[i].replace(freq_word, '')
                        clients[i] = ' '.join(clients[i].split())

        results[tier] = clients
        
    for i in range(len(results['url'])-1, -1, -1):
        if results['alt'][i] == results['url_tail'][i] == results['common'][i] == results['common'][i] == '':
            results['url'] = results['url'][:i] + results['url'][i+1:]
            results['page'] = results['page'][:i] + results['page'][i+1:]
            results['alt'] = results['alt'][:i] + results['alt'][i+1:]
            results['url_tail'] = results['url_tail'][:i] + results['url_tail'][i+1:]
            results['common'] = results['common'][:i] + results['common'][i+1:]

    return results

In [ ]:
def client_path_from_company(company):
    return f'HTML Text/{company}/{company.lower()}-clients.json'

In [ ]:
def print_clients(results):
    for i in range(len(results[list(results.keys())[0]])):
        print('[1]', results['alt'][i])
        print('[2]', results['url_tail'][i])
        print('[3]', results['common'][i])
        print()

In [ ]:
results = json_to_clients_2(client_path_from_company('Toposens'))
print_clients(results)

In [ ]:
html_text_dir = 'HTML Text'
all_results = results = {
    'url': [],
    'page': [],
    'alt': [],
    'url_tail': [],
    'common': []
}

for company in os.listdir(html_text_dir):
    results = json_to_clients_2(client_path_from_company(company))
    all_results['url'].extend(results['url'])
    all_results['page'].extend(results['page'])
    all_results['alt'].extend(results['alt'])
    all_results['url_tail'].extend(results['url_tail'])
    all_results['common'].extend(results['common'])
    print_clients(results)
    print()

client_df = pd.DataFrame(all_results)
client_df.to_csv('clients.csv', index=False, encoding='utf-8-sig')

+--------------+
| infsoft gmbh |
+--------------+
[1] frankfurt airport
[2] frankfurt airport
[3] frankfurt airport

[1] siemens
[2] siemens
[3] siemens

[1] F. Hoffmann La Roche
[2] roche
[3] roche

[1] siemens healthineers
[2] siemens healthineers
[3] healthineers siemens

[1] swisscom
[2] swisscom
[3] swisscom

[1] UNIDO
[2] vereinte nationen
[3] ap_human_geography


+----------+
| meridian |
+----------+

+----------+
| navisens |
+----------+

+-----------+
| lazarillo |
+-----------+
Duplicate: press
[1] Tampa Bay Wave
[2] support
[3] support Tampa Wave Bay

[1] Magical Startups
[2] support
[3] Startups support Magical

[1] Open Beauchef
[2] support
[3] Beauchef support Open

[1] ISCI
[2] support
[3] support ISCI

[1] corfo
[2] corfo
[3] corfo

[1] Tampa Bay Times
[2] tampabay
[3] tampa_bay_times

[1] Fox 13 News
[2] fox
[3] press News Fox 13

[1] Telemundo
[2] telemundo
[3] telemundo

[1] Tampa Bay 10
[2] tampa
[3] press Tampa Bay 10

[1] 83 Degrees
[2] 
[3] degrees 83


+-----

In [ ]:
for company in os.listdir(html_text_dir)[:1]:
    for file in os.listdir(os.path.join(html_text_dir, company)):
        if file.endswith('clients.json'):
            try:
                print(company)
                clients = json_to_clients_1(os.path.join(html_text_dir, company, file))
                print()
            except SyntaxError:
                pass

NameError: ignored